In [1]:
import pandas as pd
import numpy as np
import glob
import numpy.core.multiarray
#import numpy.core._multiarray_umath
'''from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping'''
print('2')
df = pd.read_csv("C:\\Users\\soham\\Documents\\Deep_Learning\\Fannie-Mae\\data\\join.csv")
df.shape

2


(3652611, 57)

In [2]:
df.shape
df.dtypes
#df["firstTimeHomeBuyer"].value_counts()

Unnamed: 0                              int64
origChannel                            object
sellerName                             object
origIntRate                           float64
orgUPB                                  int64
orgLoanTerm                             int64
orginationDate                         object
firstPaymentDate                       object
orgLTV                                float64
orgCombinedLTV                        float64
numBorrowers                          float64
orgDebttoIncomeRatio                  float64
borrowerCreditScoreAtOrigination      float64
firstTimeHomeBuyer                     object
loanPurpose                            object
propType                               object
numUnits                                int64
occType                                object
propertyState                          object
zipCodeShort                            int64
priMortgageInsurancePercent           float64
prodType                          

In [13]:
dx = df['currLoanDelinquencyStatus']
print(dx.shape)

KeyError: 'currLoanDelinquencyStatus'

In [3]:
from sklearn.preprocessing import LabelEncoder

df = df.drop(["Unnamed: 0","Unnamed: 0.1","loanID"], axis = 1)

obj_df = df.select_dtypes(include=['object']).copy()
lb_make = LabelEncoder()
for column in obj_df:  
    df[column] = lb_make.fit_transform(df["propertyState"])
    df[column].head(11)

In [7]:
df.head()
df = df.dropna(axis=1, how='all')

In [8]:
df.to_csv("C:\\Users\\soham\\Documents\\Deep_Learning\\Fannie-Mae\\data\\final2.csv")

In [6]:
dx = pd.read_csv("C:\\Users\\soham\\Documents\\Deep_Learning\\Fannie-Mae\\data\\final2.csv")

In [ ]:
from numpy import array
upperLimit = int(df.shape[0]*0.9)
trainData = df[:upperLimit]


train_x = trainData.drop(columns = ["currLoanDelinquencyStatus"]) #title of column that says whether they defaulted

train_y = to_categorical(trainData.currLoanDelinquencyStatus)
model = Sequential()
n_cols = train_x.shape[1]

model.add(Dense(250, activation = "relu", input_shape = (n_cols,)))
model.add(Dense(200, activation = "relu", input_shape = (n_cols,)))
model.add(Dense(150, activation = "relu", input_shape = (n_cols,)))
model.add(Dense(100, activation = "relu", input_shape = (n_cols,)))
model.add(Dense(50, activation = "relu", input_shape = (n_cols,)))

model.add(Dense(2, activation = "softmax"))

model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

early_stopping_monitor = EarlyStopping(patience = 5)

model.fit(train_x, train_y, batch_size = 500, epochs = 50, validation_split=0.1, callbacks=[early_stopping_monitor])

def predictModel(L):
  #L = [currIntRate, currActualUPB, loanAge, remainingMonthsToLegalMaturity, origIntRate, orgUPB, orgLoanTerm, orgCombinedLTV, 
  #      numBorrowers, orgDebttoIncomeRatio, borrowerCreditScoreAtOrigination, firstTimeHomeBuyer, loanPurpose, occType]
  return model.predict(array([L]))
test_data = df[upperLimit:]
test_x = test_data.drop(columns = ["currLoanDelinquencyStatus"])
print(test_x[0:10])
test_y_predictions = predict(test_x)
#test_y_predictions = predictModel([1,2,1,2,1,2,1,2,1,2,1,2,1,2])
print(test_y_predictions)

numCorrect = 0
numWrong = 0
#print(df)
#print(upperLimit)
bad = 0
for i in range(0, df.shape[0]-upperLimit):
  if test_y_predictions[i][1] > 0.5:
    if df.at[i,"currLoanDelinquencyStatus"] == 1:
      numCorrect += 1
      #bad += 1
    else:
      numWrong += 1
  else:
    if df.at[i,"currLoanDelinquencyStatus"] == 1:
      numWrong += 1
    else:
      numCorrect += 1
print("Number of wrong guesses:", numWrong)
print("Number of correct guesses:", numCorrect)
#print("Bad", bad)
def pred(currIntRate= 0, currActualUPB= 0, loanAge= 0, remainingMonthsToLegalMaturity= 0, origIntRate= 0, orgUPB= 0, orgLoanTerm= 0, orgCombinedLTV= 0, 
        numBorrowers= 0, orgDebttoIncomeRatio= 0, borrowerCreditScoreAtOrigination= 0, firstTimeHomeBuyer= 0, loanPurpose= 0, occType= 0):
  return model.predict(array([currIntRate, currActualUPB, loanAge, remainingMonthsToLegalMaturity, origIntRate, orgUPB, orgLoanTerm, orgCombinedLTV, 
        numBorrowers, orgDebttoIncomeRatio, borrowerCreditScoreAtOrigination, firstTimeHomeBuyer, loanPurpose, occType]))

In [ ]:
currIntRate= #current interest rate
currActualUPB= #current actual outstanding unpaid principal balance of a mortgage loan
loanAge= #number of months since the start of the loan
remainingMonthsToLegalMaturity= #Number of months until loan should be paid out completely
origIntRate= #original interest rate
orgUPB= #original actual outstanding unpaid principal balance of a mortgage loan
orgLoanTerm= #number of months in which regularly scheduled payments are to be made
orgCombinedLTV= #ratio calculated for a mortgage loan dependent on the loan to ratio value
numBorrowers= #number of individuals obligated to repay the loan
orgDebttoIncomeRatio= #borrower's total monthly expenses divided by total monthly income
borrowerCreditScoreAtOrigination= #borrower's credit score when the loan is taken
firstTimeHomeBuyer= #1 if is a first time home buyer, 0 if is not or is unknown
loanPurpose= #2 if loan for purchase, 1 if loan for cash out refinance, 0 if loan for no cash out refinance or is unknown
occType= #3 if for a primary home, 2 if for a secondary home, 1 if for investing, 0 for unknown purpose
test_y_predictions = predictModel([[currIntRate, currActualUPB, loanAge, remainingMonthsToLegalMaturity, origIntRate, orgUPB, orgLoanTerm, orgCombinedLTV, numBorrowers, orgDebttoIncomeRatio, borrowerCreditScoreAtOrigination, firstTimeHomeBuyer, loanPurpose, occType]])